In [10]:
import requests
from bs4 import BeautifulSoup
import json
import re
from datetime import datetime
import pandas as pd

In [2]:
def get_ptt_article(url,headers):
    try:
        print('--->url: %s'%(url))
        doc = dict()
        res = requests.get(url,headers=headers)
        soup = BeautifulSoup(res.text,'lxml')
        meta_values = soup.findAll('span',attrs={'class':'article-meta-value'})
        author = meta_values[0].text
        author_id,author_nickname = re.match('(.+) \((.*)\)',author).group(1),re.match('(.+) \((.*)\)',author).group(2)
        board = meta_values[1].text
        title = meta_values[2].text
        dt = meta_values[3].text
        tm = int(datetime.strptime(dt,'%a %b %d %H:%M:%S %Y').timestamp())
        author_infos = soup.findAll('span',attrs={'class':'f2'})
        for span in author_infos:
            m = re.match('※ 發信站\: 批踢踢實業坊\(ptt\.cc\), 來自\: (\d.+)',span.text) 
            if m:
                ip = m.group(1)
        pushs = soup.findAll('div',attrs={'class':'push'})
        comments = [{
           'tag': push.find('span',attrs={'class':'push-tag'}).text.strip(),
           'author': push.find('span',attrs={'class':'push-userid'}).text.strip(),
           'content': push.find('span',attrs={'class':'push-content'}).text.split(':')[1].strip(),
           'tm': int(datetime.strptime( '%s/%s'%(datetime.strptime(dt,'%a %b %d %H:%M:%S %Y').year,push.select('.push-ipdatetime')[0].text.strip()),'%Y/%m/%d %H:%M').timestamp())
         } for push in pushs]
        content_obj = soup.find('div',{'id':'main-content'})
        for s in content_obj.findAll('span',{'class':'f2'}):
            s.decompose()

        for s in content_obj.findAll('span',{'class':'article-meta-tag'}):
            s.decompose()

        for s in content_obj.findAll('span',{'class':'article-meta-value'}):
            s.decompose()

        for s in content_obj.findAll('div',{'class':'push'}):
            s.decompose()
        content = content_obj.text.strip()
        content = content[:-2].strip() if content[-2:] == '--' else content
        doc['author_id'] = author_id
        doc['author_nickname'] = author_nickname
        doc['board'] = board
        doc['tm'] = tm
        doc['ip'] = ip
        doc['comments'] = comments
        doc['title'] = title
        doc['content'] = content
    except Exception as e:
        print('%s has error '%(url))
        print(e)
    return doc

In [8]:
def get_ptt_article_list(page_url,headers):
    urls = []
    domain = 'https://www.ptt.cc'
    res = requests.get(page_url,headers=headers)
    soup = BeautifulSoup(res.text,'lxml')
    article_list = soup.select('.r-ent .title a')
    for article in article_list:
        url = domain + article['href']
#         print(url)
        urls.append(url)
    return urls

In [4]:
def get_ptt(board,total_page=1):
    docs = []
    headers = {
      "cookie": "__cfduid=d5f2274637142c63b85585c60c8fa1e9b1502083011; _ga=GA1.2.2014264793.1502083012; over18=1; _gid=GA1.2.504188955.1532926138",\
      "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36"\
    }
    domain = 'https://www.ptt.cc'
    url = '%s/bbs/%s/index.html'%(domain,board)
    print(url)
    res =requests.get(url,headers=headers)
    soup = BeautifulSoup(res.text,'lxml')
    for i in soup.select('.btn.wide'):
        if i.text == '‹ 上頁':
            m = re.match('.+/index(\d+).html',i['href'])
            last_page = int(m.group(1))+1
    print(last_page)
    for page in range(last_page,last_page-total_page,-1):
        page_url = '{}/bbs/{}/index{}.html'.format(domain,board,page)
        urls =get_ptt_article_list(page_url,headers)
        for url in urls:
            doc = get_ptt_article(url,headers)
            docs.append(doc)
    df = pd.DataFrame(list(filter(None,docs)))
    return df

In [11]:
get_ptt('baseball')

https://www.ptt.cc/bbs/baseball/index.html
8377
--->url: https://www.ptt.cc/bbs/Baseball/M.1537439050.A.C77.html
--->url: https://www.ptt.cc/bbs/Baseball/M.1537439932.A.387.html
--->url: https://www.ptt.cc/bbs/Baseball/M.1537441057.A.5B7.html
--->url: https://www.ptt.cc/bbs/Baseball/M.1537442144.A.21C.html
--->url: https://www.ptt.cc/bbs/Baseball/M.1537442494.A.82C.html
--->url: https://www.ptt.cc/bbs/Baseball/M.1537442696.A.D1C.html
--->url: https://www.ptt.cc/bbs/Baseball/M.1537443413.A.23B.html
--->url: https://www.ptt.cc/bbs/Baseball/M.1537444060.A.8DE.html
--->url: https://www.ptt.cc/bbs/Baseball/M.1537445106.A.1A5.html
--->url: https://www.ptt.cc/bbs/Baseball/M.1537445106.A.214.html
--->url: https://www.ptt.cc/bbs/Baseball/M.1537445121.A.8CC.html
--->url: https://www.ptt.cc/bbs/Baseball/M.1537445181.A.654.html
--->url: https://www.ptt.cc/bbs/Baseball/M.1537446297.A.497.html
--->url: https://www.ptt.cc/bbs/Baseball/M.1514858845.A.576.html
https://www.ptt.cc/bbs/Baseball/M.15148588

,author_id,author_nickname,board,comments,content,ip,title,tm
0,elephant2006,泰謝拉,Baseball,"[{'tag': '噓', 'author': 'liaogary', 'content':...",去年季後賽把一堆好手丟二軍\n\n放一堆菜比八來練兵羞辱觀眾\n\n這個球季看起來也是在練兵...,101.15.152.177,[討論] 中信兄弟是不是留一手？,1537439047
1,thnlkj0665,灰色地帶,Baseball,"[{'tag': '→', 'author': 'newz', 'content': '收編...",中職／選秀狀元戴培峰一軍隨隊 林哲瑄：不要慌 \n\n記者陳立勳／台南報導\n\n富邦悍將本...,118.150.84.5,[新聞] 選秀狀元戴培峰一軍隨隊 林哲瑄：不要慌,1537439929
2,jogging23,忠言逆耳,Baseball,"[{'tag': '推', 'author': 'KimuraBill', 'content...",肥恰狀況爛成這鳥樣還場場先發扛四棒\n志豪可以拉下先發為何肥恰不能？\n林32上周明明狀況不...,180.217.91.221,[討論] 為何肥恰可以一直排第四棒,1537441054
3,Noemi1106,超級喜歡,Baseball,"[{'tag': '推', 'author': 'stonys', 'content': '...",http://se.piee.pw/AKYVE \n\n剛剛啊\n\n周思齊一支安打\n\n...,101.15.231.253,[討論] 江大裁判是恍神還是針對誰？,1537442141
4,zzyyxx77,討噓大師77,Baseball,"[{'tag': '推', 'author': 'Zuleta', 'content': '...",史奈爾身手比美普萊斯 有望低調抱走塞揚獎\n\n提到光芒隊史單季投手最佳表現，絕非2012年...,61.228.25.233,[新聞] 史奈爾身手比美普萊斯 有望低調抱走塞揚,1537442489
5,pchunters123,pp表接受。和感謝，「今,Baseball,"[{'tag': '→', 'author': 'dunabs', 'content': '...",中信無緣戰績第一 想打季後賽要自立自強\n\n中信兄弟昨天以1比5敗給Lamigo桃猿隊後，...,180.217.92.233,[新聞] 中信無緣戰績第一 想打季後賽要自立自強,1537442694
6,mjdrcb,annin,Baseball,"[{'tag': '→', 'author': 'hicker', 'content': '...",海峽棒球賽／張泰山3安猛打賞帶頭 台壽保2連勝\n\nhttps://tinyurl.com...,1.164.200.13,[新聞] 海峽棒球賽／張泰山3安猛打賞帶頭 台壽保,1537443408
7,Boonen,Tommeke,Baseball,"[{'tag': '推', 'author': 'Zuleta', 'content': '...",剛剛在爬小聯盟球員的近況，突然想起這位跟基哥同期的捕手\n\n有人知道武昭關後來去哪了嗎？\...,36.235.211.5,Re: [討論] 武昭關大聯盟夢碎 可望加入中職,1537444059
8,saiulbb,Becky♪＃是我的拉!,Baseball,"[{'tag': '推', 'author': 'LastDinosaur', 'conte...",1. 新聞來源：\n\nTSNA\n\n2. 新聞標題：\n\n響尾蛇咬傷小熊止敗 Lovu...,61.56.143.149,[新聞] 響尾蛇咬傷小熊止敗 Lovullo：沒有認輸,1537445100
9,saiulbb,Becky♪＃是我的拉!,Baseball,[],1. 新聞來源：\n\nTSNA\n\n2. 新聞標題：\n\n開轟為勇士止敗 Freema...,61.56.143.149,[新聞] 開轟為勇士止敗 Freeman享受贏球,1537445101
